<h1 align="center">Python Data Science Guides</h1>
<h2 align="center">Pandas - Data Cleaning</h2>

&nbsp;

### Contents

Section 1 - Handling Missing Values

Section 2 - Converting Data Types

Conclusion

<h2 align="center">Section 1 - Handling Missing Values</h2>


### 1.1 - Missing Values in Pandas

The airline data used in previous notebooks has been reduced down to 50 passengers, with some data removed to create missing values. This is more representative of real-world data, where there may not be a complete collection of information for every individual in a population.

Missing values are typically referred to a NaN values, which is short for *not a number*. NumPy uses `NaN` as a missing value, and since Pandas is built on top of NumPy this can be used in Pandas natively. To assign NaN to an element or variable however, NumPy must be imported. Technically `numpy.nan` is a float, and so care must be taken when casting data types (this is covered later in the notebook). Missing data is handled differently depending on the situation. For example, the most simple method to deal with missing value is to simply remove any rows that contain them. If the data being studied is more limited and expensive to acquire, then imputation may be the best course of action - which is a series of methods for filling in the missing data based on calculations or best guesses.

By default, Pandas will treat any of the following as missing values when reading data:

|            |           |        |
|------------|-----------|--------|
| ‘’         | ‘-NaN’    | ‘NA’   |
| ‘#N/A’     | ‘-nan’    | ‘NULL’ |
| ‘#N/A N/A’ | ‘1.#IND’  | ‘NaN’  |
| ‘#NA’      | ‘1.#QNAN’ | ‘n/a’  |
| ‘-1.#IND’  | ‘< NA >’  | ‘nan’  |
| ‘-1.#QNAN’ | ‘N/A’     | ‘null’ |

<style>
table,td,tr,th {border:none!important}
</style>

### 1.2 - Handling Missing Values when Reading from a File

Sometimes missing values are marked with a symbol that is not in the list of missing value symbols shown above. Because of this Pandas will not recognise that data is missing and will simply read in the data as-is. The dataset used in this example uses a mixture of blank elements (which are included in the Pandas list of missing values) and '-' symbols (which are not included in the Pandas list of missing values). For this reason, the '-' symbols need to be handled explicitly when reading the file. This can be achieved by passing a list of strings to the `na_values` parameter for each value that should be treated as a `NaN`.


In [206]:
import pandas as pd

# Read in the csv without handling '-' as a missing value
df = pd.read_csv('airline_edited.csv')
df.head()

,ID,Gender,Age,Customer Type,Type of Travel,Class,Flight Distance,Departure Delay,Arrival Delay
0,1,-,48,First-time,Business,Business,NaN,2.0,5.0
1,2,Female,35,Returning,-,Business,821.0,NaN,39.0
2,3,Male,41,NaN,Business,Business,853.0,0.0,0.0
3,4,Male,50,Returning,Business,Business,1905.0,0.0,0.0
4,5,Female,49,Returning,Business,Business,3470.0,0.0,1.0


In [207]:
# Read in the csv again and handle '-' as a missing value
df = pd.read_csv('airline_edited.csv', na_values=['-'])
df.head()

,ID,Gender,Age,Customer Type,Type of Travel,Class,Flight Distance,Departure Delay,Arrival Delay
0,1,NaN,48,First-time,Business,Business,NaN,2.0,5.0
1,2,Female,35,Returning,NaN,Business,821.0,NaN,39.0
2,3,Male,41,NaN,Business,Business,853.0,0.0,0.0
3,4,Male,50,Returning,Business,Business,1905.0,0.0,0.0
4,5,Female,49,Returning,Business,Business,3470.0,0.0,1.0


### 1.3 - Count the Number of Missing Values in a DataFrame or Series using `isna()` and `sum()`

The `isna` method returns a Boolean mask, which is copy of the object (either DataFrame or Series) where the values are replaced with `True` if the data is missing in that cell, and `False` if it is not. This method can be combined with the `sum` method, to add the number of `True` values that appear in the mask, thus giving a count for the number of missing values in the object. For DataFrames, the `axis` parameter can be used in the `sum` method to change the direction the sum acts across. By default, the sum will add the number of `True` values along each column (`axis=0`), but this can be changed to determine the number of missing values in each row by passing `axis=1`.

Note that a blank string in a DataFrame/Series is not counted as a `NaN` value. Blank strings are however converted to `NaN` values when read directly from a file, and so in this case the value will be a `NaN`.

The `isna` method is a newer version of the `isnull` method, and so this may still be seen in older code. More information can be found in the [documentation](https://pandas.pydata.org/docs/reference/api/pandas.isna.html).

In [208]:
df.isna().head()

,ID,Gender,Age,Customer Type,Type of Travel,Class,Flight Distance,Departure Delay,Arrival Delay
0,False,True,False,False,False,False,True,False,False
1,False,False,False,False,True,False,False,True,False
2,False,False,False,True,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False


In [209]:
# Find the number of NaN values in each column
df.isna().sum()

ID                 0
Gender             4
Age                0
Customer Type      7
Type of Travel     3
Class              5
Flight Distance    4
Departure Delay    6
Arrival Delay      1
dtype: int64

In [210]:
# Find the number of NaN values in each row
df.isna().sum(axis=1).head()

0    2
1    2
2    1
3    0
4    0
dtype: int64

### 1.4 - Fill in Missing Values with `fillna()`

If all the missing values can be replaced with a single common value, the `fillna` method can be used. This takes an argument for the value to fill each missing value with and returns a DataFrame/Series with the values filled. The example below fills each `NaN` value with the string 'NO DATA'. There are also a number of optional arguments which are described below, and more information can be found in the [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html).

|            |                                                                                                                                                                                                                                                                                                                                                                 |
|------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| *value*    | The value to replace each `NaN` value with. This is required if *method* is `None`.                                                                                                                                                                                                                                                                             |
| *method*   | The method for replacing `NaN` values. The options are 'backfill'/'bfill' and 'pad'/'ffill'. If 'backfill' or 'bfill' are selected then the next valid entry in the column will be backpropagated to fill the missing value. If 'pad' or'ffill' are selected, then the previous valid entry will be propagated forwards. This is required if *value* is `None`. |
| *axis*     | The axis along which to fill missing values, defaults to 0 for columns but can be passed 1 for rows (to be used in conjunction with *method*).                                                                                                                                                                                                                  |
| *inplace*  | A Boolean which if `True` will overwrite the current object with the changes, and if `False` will not. The default value is `False`.                                                                                                                                                                                                                            |
| *limit*    | An integer limit on the number of backpropagated/forwardpropagated values that can be used to fill missing data. This can only be used when the *method* argument is not `None`.                                                                                                                                                                                |
| *downcast* | A dictionary of which items should be cast to different data types where possible during the filling process.                                                                                                                                                                                                                                                   |

<style>
table,td,tr,th {border:none!important}
</style>

In [211]:
df.fillna('NO DATA').head()

,ID,Gender,Age,Customer Type,Type of Travel,Class,Flight Distance,Departure Delay,Arrival Delay
0,1,NO DATA,48,First-time,Business,Business,NO DATA,2.0,5.0
1,2,Female,35,Returning,NO DATA,Business,821.0,NO DATA,39.0
2,3,Male,41,NO DATA,Business,Business,853.0,0.0,0.0
3,4,Male,50,Returning,Business,Business,1905.0,0.0,0.0
4,5,Female,49,Returning,Business,Business,3470.0,0.0,1.0


### 1.5 - Replace Missing Values with `replace()`

If the data has already been read into a DataFrame and *na_values* argument was not passed, there may be strings in the DataFrame that needed to be replaced with `NaN` values. Recall that a `NaN` value can be passed using `numpy.nan`. To demonstrate this, the data will loaded again but this time without passing the *na_values* argument. 

The `replace` method can be used to replace values in a DataFrame/Series object. The first parameter, `to_replace`, is the value to be replaced, and the second, `value`, is the value that will be used to make the replacement. A description of the parameters is given below, and more detail can be found in the [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html).

|            |                                                                                                                                                                                                                                                                                                                                                                                                                          |
|------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| to_replace | A string, int, float, regular expression, list of values, dictionary of values, or Series of values to be replaced. If a dictionary is passed then this must contain the values to replace as the keys and the replacement values as the values.                                                                                                                                                                         |
| value      | A string, int, float, Series, dict, list, or regular expression containing the replacement value(s).                                                                                                                                                                                                                                                                                                                     |
| inplace    | A Boolean which if `True` will overwrite the current object with the changes, and if `False` will not. The default value is `False`.                                                                                                                                                                                                                                                                                     |
| limit      | An integer of the length of a gap to backfill or forwardfill.                                                                                                                                                                                                                                                                                                                                                            |
| regex      | A Boolean which if `True` will treat the arguments for *to_replace* and/or *value* as a regular expression and if `False` will not.                                                                                                                                                                                                                                                                                      |
| method     | The method to use to replace values when *to_replace* is not a dictionary and *value* is `None`. The options are 'backfill'/'bfill' and 'pad'/'ffill'. If 'backfill' or 'bfill' are selected then the next valid entry in the column will be backpropagated to replace the old value. If 'pad' or'ffill' are selected, then the previous valid entry will be propagated forwards. This is required if *value* is `None`. |

<style>
table,td,tr,th {border:none!important}
</style>

In [212]:
# Read in the data without replacing the '-' characters with NaN values
df = pd.read_csv('airline_edited.csv')
df.head(3)

,ID,Gender,Age,Customer Type,Type of Travel,Class,Flight Distance,Departure Delay,Arrival Delay
0,1,-,48,First-time,Business,Business,NaN,2.0,5.0
1,2,Female,35,Returning,-,Business,821.0,NaN,39.0
2,3,Male,41,NaN,Business,Business,853.0,0.0,0.0


In [213]:
# Replace '-' with NaN values
import numpy as np
df.replace('-',np.nan, inplace=True)
df.head(3)

,ID,Gender,Age,Customer Type,Type of Travel,Class,Flight Distance,Departure Delay,Arrival Delay
0,1,NaN,48,First-time,Business,Business,NaN,2.0,5.0
1,2,Female,35,Returning,NaN,Business,821.0,NaN,39.0
2,3,Male,41,NaN,Business,Business,853.0,0.0,0.0


In [214]:
# Replacing multiple values with a list
df.replace(['First-time','Business'], ['FT','BN']).head(3)

,ID,Gender,Age,Customer Type,Type of Travel,Class,Flight Distance,Departure Delay,Arrival Delay
0,1,NaN,48,FT,BN,BN,NaN,2.0,5.0
1,2,Female,35,Returning,NaN,BN,821.0,NaN,39.0
2,3,Male,41,NaN,BN,BN,853.0,0.0,0.0


### 1.6 - Identifying Missing Values using `pd.unique()`

Sometimes a dataset will contain strings to identify missing values which are non-standard. For example, the string 'missing' could be used instead of 'NaN', or 'no data' etc. It can be useful to view a list of all the unique values in a 1-D object (such as a Series) to identify which values should be acted upon using the `replace` method. This can be achieve with the `unique` method which takes a single required argument for the data to work with.


In [215]:
# View the unique values in the Customer Type column
pd.unique(df['Customer Type'])

array(['First-time', 'Returning', nan], dtype=object)

### 1.7 - Count the Number of Non-Missing Values in a DataFrame or Series using `notna()` and `sum()`
The `notna` method does the opposite of the `isna` method and returns a Boolean mask of a DataFrame/Series. If data is not missing the cell is replaced with `True`, and if it is the cell is replaced with `False`. This can be useful when combined with the `sum` method to determine the number of non-missing values. Again, the `axis` parameter can be set using 0 for columns and 1 for rows to sum in difference axes of a DataFrame.

The `notna` method is a newer version of the `notnull` method, and so this may still be seen in older code. More information can be found in the [documentation](https://pandas.pydata.org/docs/reference/api/pandas.notna.html).

In [216]:
df.notna().head()

,ID,Gender,Age,Customer Type,Type of Travel,Class,Flight Distance,Departure Delay,Arrival Delay
0,True,False,True,True,True,True,False,True,True
1,True,True,True,True,False,True,True,False,True
2,True,True,True,False,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True


In [217]:
# Find the number of non-NaN values in each column
df.notna().sum()

ID                 50
Gender             46
Age                50
Customer Type      43
Type of Travel     47
Class              45
Flight Distance    46
Departure Delay    44
Arrival Delay      49
dtype: int64

In [218]:
# Find the number of non-NaN values in each row
df.notna().sum(axis=1).head()

0    7
1    7
2    8
3    9
4    9
dtype: int64

### 1.8 - Drop Missing Values with `dropna()`

In some cases it is more appropriate to simply remove an entire row if it contains a `NaN` value in any of its columns. This is simple to achieve in Pandas and can be performed using the `dropna` method. There are a few optional arguments
which are described below. More information on this method can be found [here](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html).

|           |                                                                                                                                                                                                                                                                    |
|-----------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| *axis*    | The axis to drop if `NaN` value(s) are found. 0 or 'index' for rows and 1 or 'columns' for columns. If `None` the default is 0.                                                                                                                                    |
| *how*     | How to decide if a row/column should be dropped. Acceptable values are 'any' and 'all'. If 'any' drop the row/column if at least one `NaN` value is found. If 'all' only drop the row/column if the entire Series is `NaN` values. If `None` the default is 'any'. |
| *thresh*  | An integer number of non-`NaN` values required for a row/column to be kept. The row/column will be dropped if more values are missing than the threshold.                                                                                                                                                                                         |
| *subset*  | A list of columns (or rows if dropping columns) to consider when dropping rows. If `NaN` values are permissible in one row or column, then this should be left out of the list.                                                                                    |
| *inplace* | A Boolean which if `True` will overwrite the current object with the changes, and if `False` will not. The default value is `False`.                                                                                                                               |

<style>
table,td,tr,th {border:none!important}
</style>

In [219]:
# Drop the rows with any missing data, the rows 1, 2, 5, 6, 8 and 9 can be seen to be missing in this DataFrame
df.dropna().head()

,ID,Gender,Age,Customer Type,Type of Travel,Class,Flight Distance,Departure Delay,Arrival Delay
3,4,Male,50,Returning,Business,Business,1905.0,0.0,0.0
4,5,Female,49,Returning,Business,Business,3470.0,0.0,1.0
7,8,Female,60,Returning,Business,Business,853.0,0.0,3.0
10,11,Female,28,First-time,Business,Business,821.0,0.0,5.0
11,12,Female,27,First-time,Business,Business,421.0,20.0,21.0


In [220]:
# Drop the rows with where the entire row is missing data, note that no rows have been dropped
df.dropna(how='all').head()

,ID,Gender,Age,Customer Type,Type of Travel,Class,Flight Distance,Departure Delay,Arrival Delay
0,1,NaN,48,First-time,Business,Business,NaN,2.0,5.0
1,2,Female,35,Returning,NaN,Business,821.0,NaN,39.0
2,3,Male,41,NaN,Business,Business,853.0,0.0,0.0
3,4,Male,50,Returning,Business,Business,1905.0,0.0,0.0
4,5,Female,49,Returning,Business,Business,3470.0,0.0,1.0


In [221]:
# Drop rows where NaN values are found in the Type of Travel column only
df.dropna(subset='Type of Travel').head()

,ID,Gender,Age,Customer Type,Type of Travel,Class,Flight Distance,Departure Delay,Arrival Delay
0,1,NaN,48,First-time,Business,Business,NaN,2.0,5.0
2,3,Male,41,NaN,Business,Business,853.0,0.0,0.0
3,4,Male,50,Returning,Business,Business,1905.0,0.0,0.0
4,5,Female,49,Returning,Business,Business,3470.0,0.0,1.0
5,6,NaN,43,Returning,Business,Business,3788.0,NaN,0.0


In [222]:
# Drop rows where both Gender and Age are missing (not both are required, but at least one must be present)
df.dropna(how='all', subset=['Gender','Age']).head()

,ID,Gender,Age,Customer Type,Type of Travel,Class,Flight Distance,Departure Delay,Arrival Delay
0,1,NaN,48,First-time,Business,Business,NaN,2.0,5.0
1,2,Female,35,Returning,NaN,Business,821.0,NaN,39.0
2,3,Male,41,NaN,Business,Business,853.0,0.0,0.0
3,4,Male,50,Returning,Business,Business,1905.0,0.0,0.0
4,5,Female,49,Returning,Business,Business,3470.0,0.0,1.0


In [223]:
# Drop rows that more than one column with a NaN value
# There are 9 columns, a so a row must have at least 9-1 (which is 8) values to be kept
df.dropna(thresh=len(df.columns)-1).head()

,ID,Gender,Age,Customer Type,Type of Travel,Class,Flight Distance,Departure Delay,Arrival Delay
2,3,Male,41,NaN,Business,Business,853.0,0.0,0.0
3,4,Male,50,Returning,Business,Business,1905.0,0.0,0.0
4,5,Female,49,Returning,Business,Business,3470.0,0.0,1.0
6,7,Male,43,Returning,Business,Business,1963.0,NaN,0.0
7,8,Female,60,Returning,Business,Business,853.0,0.0,3.0


<h2 align="center">Section 2 - Converting Data Types</h2>

### 2.1 - Select Columns by Data Type using `select_dtypes()`

Sometimes it is useful to select columns which contain certain data types. For example, selecting only the numerical columns from a dataset might be useful to take forward for calculations. There are a number of different data types which can be selected using the `select_dtypes` method, which are listed below:

* int64 - NumPy 64-bit integers (the integer type Pandas uses by default), int32, int16 and int8 are also accepted.
* float64 - see above except for floats, note that `numpy.nan` is regarded as a float.
* number - any numeric data type including integers and floats.
* object - strings or mixed data types in the same column.
* category - a categoric data type in Pandas, similar to a string.
* datetime - datetime objects, can also be referenced as datetime64.
* timedelta - timedelta objects, can also be referenced as timedelta64.
* bool - Boolean values.

The method only takes two arguments: *include* and *exclude*. The *include* parameter accepts a data type or list of data types to include in the search, while exclude accepts a data type or list of data types to exclude from the search.

In [224]:
# Get a unique set of the data types in the DataFrame
set(df.dtypes)

{dtype('int64'), dtype('float64'), dtype('O')}

In [225]:
# Select the columns containing only numeric data
df.select_dtypes(include='number').head(3)

,ID,Age,Flight Distance,Departure Delay,Arrival Delay
0,1,48,NaN,2.0,5.0
1,2,35,821.0,NaN,39.0
2,3,41,853.0,0.0,0.0


In [226]:
# Select the columns containing object data (all strings or mixed data types)
df.select_dtypes(include='object').head(3)

,Gender,Customer Type,Type of Travel,Class
0,NaN,First-time,Business,Business
1,Female,Returning,NaN,Business
2,Male,NaN,Business,Business


In [227]:
# Select the columns containing numeric or object data (which is every column in this example)
df.select_dtypes(include=['number','object']).head(3)

,ID,Gender,Age,Customer Type,Type of Travel,Class,Flight Distance,Departure Delay,Arrival Delay
0,1,NaN,48,First-time,Business,Business,NaN,2.0,5.0
1,2,Female,35,Returning,NaN,Business,821.0,NaN,39.0
2,3,Male,41,NaN,Business,Business,853.0,0.0,0.0


In [228]:
# Select the columns that do not contain int64 data
df.select_dtypes(exclude='int64').head(3)

,Gender,Customer Type,Type of Travel,Class,Flight Distance,Departure Delay,Arrival Delay
0,NaN,First-time,Business,Business,NaN,2.0,5.0
1,Female,Returning,NaN,Business,821.0,NaN,39.0
2,Male,NaN,Business,Business,853.0,0.0,0.0


### 2.2 - Convert Data using `astype()`

The `astype` method can be used to convert DataFrames/Series from one data type to another. If the conversion is not possible, such as trying to convert alphabetical strings to integers, then an error will be returned. Note that the `numpy.nan` value is a float, and so trying to convert the string 'NaN' to an integer will give an error. A dictionary of column names and the corresponding data type they should be converted to can also be passed. as well as some other optional arguments which are described below. More information can be found [here](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.astype.html).

|                    |                                                                                                                                                                                                                                                                                                                                                                             |
|--------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| *dtype* (required) | A string of the data type to convert to or a dictionary of columns names with different data types to convert to.                                                                                                                                                                                                                                                           |
| *copy*             | A Boolean which if `True` will return a copy of the object, and will not if `False`.                                                                                                                                                                                                                                                                                        |
| *errors*           | A string specifying how to handle errors. valid options are *raise* and *ignore*. If *raise* is selected, the any errors will halt the program and the error message will be printed to the console. If *ignore* is selected, the error messages will not be displayed and the original object without casting will be returned silently. If `None` the default is *raise*. |

<style>
table,td,tr,th {border:none!important}
</style>

In [229]:
# Create a DataFrame of 3 employees, note that the final age in the Age column is a string and not a numpy.nan value
employee_dict = {'Forename': ['John','Bob','William'],
                 'Surname': ['Fry','Davison','Hamilton'],
                 'Age': ['24', '38', '25'],
                 'Salary': ['50000', '65000', 'NaN']}

employee_df = pd.DataFrame(employee_dict)
employee_df

,Forename,Surname,Age,Salary
0,John,Fry,24,50000
1,Bob,Davison,38,65000
2,William,Hamilton,25,NaN


In [230]:
# This will give an error since the string 'NaN' cannot be converted to an integer, only a float

# employee_df['Age'] = employee_df['Age'].astype('int')

In [231]:
# Convert the Series object for the Age column to floats
# This will not give an error since the string 'NaN' can be converted to the numpy.nan float
employee_df['Salary'] = employee_df['Salary'].astype('float')
employee_df

,Forename,Surname,Age,Salary
0,John,Fry,24,50000.0
1,Bob,Davison,38,65000.0
2,William,Hamilton,25,NaN


In [232]:
# Convert the entire DataFrame to strings
employee_df = employee_df.astype('string')
employee_df

,Forename,Surname,Age,Salary
0,John,Fry,24,50000.0
1,Bob,Davison,38,65000.0
2,William,Hamilton,25,<NA>


In [233]:
# Convert the columns separately (note that NaN cannot be converted to an integer so row 2 is dropped)
employee_df = employee_df.dropna()

# The data type can be written as a string
conversions = {'Age': 'int',
               'Salary': 'float'}

# The data type can also be written using a Python type key word
conversions = {'Age': int,
               'Salary': float}

employee_df = employee_df.astype(conversions)
employee_df

,Forename,Surname,Age,Salary
0,John,Fry,24,50000.0
1,Bob,Davison,38,65000.0


### 2.3 - Convert Columns using `pd.to_numeric()`

The `astype` method has a limitation in that if cannot convert one element in a DataFrame/Series an error is return and no conversion takes place. The `to_numeric` function fixes this issue by including the *errors* parameter which can force any value to a `NaN` if it cannot be converted. Note that this method only applies when converting string data to numeric data, and so is not as versatile as the `astype` method.

In the example below the salary column features a '-' to mark a missing value instead of the string 'NaN'. Using the `astype` method here would return an error. Instead, the `to_numeric` method can be used with the value *coerce* for the *errors* parameter to convert convert the '-' to a `NaN. More information on this method can be found in the [documentation](https://pandas.pydata.org/docs/reference/api/pandas.to_numeric.html).

In [234]:
# Create the DataFrame again but this time with a '-' to denote the missing value instead of 'NaN'
employee_dict = {'Forename': ['John','Bob','William'],
                 'Surname': ['Fry','Davison','Hamilton'],
                 'Age': ['24', '38', '25'],
                 'Salary': ['50000', '65000', '-']}

employee_df = pd.DataFrame(employee_dict)
employee_df

,Forename,Surname,Age,Salary
0,John,Fry,24,50000
1,Bob,Davison,38,65000
2,William,Hamilton,25,-


In [235]:
# Convert the salary column to numbers and coerce the '-' to a NaN
employee_df['Salary'] = pd.to_numeric(employee_df['Salary'], errors='coerce')
employee_df

,Forename,Surname,Age,Salary
0,John,Fry,24,50000.0
1,Bob,Davison,38,65000.0
2,William,Hamilton,25,NaN


### 2.4 - Converting Strings to Date and Time objects using `pd.to_datetime()` and 'pd.to_timedelta()'

Since the `to_numeric` method can only convert from strings to numeric values, it is more limited than the `astype` method. However, the benefits of coercing values that would usually raise an error to be `NaN` values is very useful. For that reason the `to_datetime` and `to_timedelta` methods were created which function similarly to the `to_numeric` method. More information on the two methods can be found [here](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html) for `to_datetime` and [here](https://pandas.pydata.org/docs/reference/api/pandas.to_timedelta.html) for `to_timedelta`.

&nbsp;

### 2.5 - Use `to_numeric` on an Entire DataFrame Simultaneously using `df.apply`

The `to_numeric` method must be applied to Series, and applying to each column one-by-one in a DataFrame can be time-consuming. To overcome this, the `apply` method can be used to apply the method to every column in one line.

In [238]:
df = df.apply(pd.to_numeric, errors='coerce')
df.head()

,ID,Gender,Age,Customer Type,Type of Travel,Class,Flight Distance,Departure Delay,Arrival Delay
0,1,NaN,48,NaN,NaN,NaN,NaN,2.0,5.0
1,2,NaN,35,NaN,NaN,NaN,821.0,NaN,39.0
2,3,NaN,41,NaN,NaN,NaN,853.0,0.0,0.0
3,4,NaN,50,NaN,NaN,NaN,1905.0,0.0,0.0
4,5,NaN,49,NaN,NaN,NaN,3470.0,0.0,1.0


<h2 align="center">Conclusion</h2>

There are many ways to handle missing values in a dataset at each stage of analysis, including converting strings to `NaN` objects when reading from files, dropping missing values, replacing values, and more. The data types of data can also be easily changed, which is essential for some types of analysis. The methods shown here are extremely useful for data cleaning and preparation, which is a key aspect of data science.